In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('mytree').getOrCreate()

In [2]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import (RandomForestClassifier,
                                      GBTClassifier,
                                      DecisionTreeClassifier)

In [3]:
data = spark.read.format('libsvm').load('/FileStore/tables/sample_libsvm_data.txt')

In [4]:
data.show()

+-----+--------------------+
label| features|
+-----+--------------------+
 0.0|(692,[127,128,129...|
 1.0|(692,[158,159,160...|
 1.0|(692,[124,125,126...|
 1.0|(692,[152,153,154...|
 1.0|(692,[151,152,153...|
 0.0|(692,[129,130,131...|
 1.0|(692,[158,159,160...|
 1.0|(692,[99,100,101,...|
 0.0|(692,[154,155,156...|
 0.0|(692,[127,128,129...|
 1.0|(692,[154,155,156...|
 0.0|(692,[153,154,155...|
 0.0|(692,[151,152,153...|
 1.0|(692,[129,130,131...|
 0.0|(692,[154,155,156...|
 1.0|(692,[150,151,152...|
 0.0|(692,[124,125,126...|
 0.0|(692,[152,153,154...|
 1.0|(692,[97,98,99,12...|
 1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows

In [5]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [6]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier()
gbt = GBTClassifier()

In [7]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [8]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [9]:
gbt_preds.show()

+-----+--------------------+--------------------+--------------------+----------+
label| features| rawPrediction| probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
 0.0|(692,[95,96,97,12...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[98,99,100,1...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[123,124,125...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[124,125,126...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[125,126,127...|[1.25372151870395...|[0.92466196012994...| 0.0|
 0.0|(692,[126,127,128...|[1.41865593459653...|[0.94465909920798...| 0.0|
 0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[126,127,128...|[1.49968587065345...|[0.95254573612173...| 0.0|
 0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[126,127,128...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[127,128,129...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[153,154,155...|[1.54350200272498...|[0.95635347857270...| 0.0|
 0.0|(692,[154,155,156...|[1.25372151870395...|[0.92466196012994...| 0.0|
 0.0|(692,[154,155,156...|[-0.9516179775869...|[0.12974266629150...| 1.0|
 1.0|(692,[99,100,101,...|[1.49968587065345...|[0.95254573612173...| 0.0|
 1.0|(692,[119,120,121...|[-1.4592925012976...|[0.05124244969998...| 1.0|
 1.0|(692,[123,124,125...|[-1.5435020027249...|[0.04364652142729...| 1.0|
+-----+--------------------+--------------------+--------------------+----------+
only showing top 20 rows

In [10]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [11]:
acc_eval = MulticlassClassificationEvaluator(metricName='accuracy')

In [12]:
print('DTC Accuracy:', acc_eval.evaluate(dtc_preds))

DTC Accuracy: 0.9117647058823529

In [13]:
print('RFC Accuracy:', acc_eval.evaluate(rfc_preds))

RFC Accuracy: 0.9411764705882353

In [14]:
print('GBT Accuracy:', acc_eval.evaluate(gbt_preds))

GBT Accuracy: 0.9117647058823529

In [15]:
rfc_model.featureImportances

Out[ 23 ]: SparseVector(692, {128: 0.0016, 207: 0.0384, 216: 0.0066, 234: 0.007, 261: 0.0194, 263: 0.0068, 271: 0.0469, 289: 0.0418, 290: 0.0437, 300: 0.0063, 317: 0.0441, 322: 0.0015, 329: 0.0282, 349: 0.0234, 350: 0.0076, 351: 0.0372, 352: 0.0123, 377: 0.0917, 378: 0.05, 379: 0.0372, 399: 0.0083, 406: 0.15, 428: 0.0086, 433: 0.0033, 453: 0.0201, 460: 0.0093, 461: 0.05, 467: 0.0082, 489: 0.05, 490: 0.05, 497: 0.03, 541: 0.0034, 551: 0.0384, 554: 0.0059, 565: 0.0036, 625: 0.0091})

In [16]:
#Lets do one little more practical
data = spark.read.csv('/FileStore/tables/College.csv', inferSchema=True, header=True)

In [17]:
data.columns

Out[ 26 ]: 
['School',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate']

In [18]:
data.head(1)

Out[ 27 ]: [Row(School='Abilene Christian University', Private='Yes', Apps=1660, Accept=1232, Enroll=721, Top10perc=23, Top25perc=52, F_Undergrad=2885, P_Undergrad=537, Outstate=7440, Room_Board=3300, Books=450, Personal=2200, PhD=70, Terminal=78, S_F_Ratio=18.1, perc_alumni=12, Expend=7041, Grad_Rate=60)]

In [19]:
from pyspark.ml.feature import VectorAssembler

In [20]:
assembler = VectorAssembler(inputCols = ['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'], outputCol='features')

In [21]:
output = assembler.transform(data)

In [22]:
from pyspark.ml.feature import StringIndexer

In [23]:
indexer = StringIndexer(inputCol = 'Private', outputCol = 'PrivateIndex')

In [24]:
output_indexed = indexer.fit(output).transform(output)

In [25]:
output_indexed.printSchema()

root
-- School: string (nullable = true)
-- Private: string (nullable = true)
-- Apps: integer (nullable = true)
-- Accept: integer (nullable = true)
-- Enroll: integer (nullable = true)
-- Top10perc: integer (nullable = true)
-- Top25perc: integer (nullable = true)
-- F_Undergrad: integer (nullable = true)
-- P_Undergrad: integer (nullable = true)
-- Outstate: integer (nullable = true)
-- Room_Board: integer (nullable = true)
-- Books: integer (nullable = true)
-- Personal: integer (nullable = true)
-- PhD: integer (nullable = true)
-- Terminal: integer (nullable = true)
-- S_F_Ratio: double (nullable = true)
-- perc_alumni: integer (nullable = true)
-- Expend: integer (nullable = true)
-- Grad_Rate: integer (nullable = true)
-- features: vector (nullable = true)
-- PrivateIndex: double (nullable = false)

In [26]:
final_data = output_indexed.select('features', 'PrivateIndex')

In [27]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [28]:
from pyspark.ml.regression import RandomForestRegressor

In [29]:
from pyspark.ml import Pipeline

In [30]:
dtc = DecisionTreeClassifier(labelCol = 'PrivateIndex', featuresCol='features')
rfc = RandomForestClassifier(labelCol = 'PrivateIndex', featuresCol='features')
GBT = GBTClassifier(labelCol = 'PrivateIndex', featuresCol='features')

In [31]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = GBT.fit(train_data)

In [32]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [33]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [34]:
my_binary_eval = BinaryClassificationEvaluator(labelCol = 'PrivateIndex')

In [35]:
print('DTC')
print('My binary classification accuracy:', my_binary_eval.evaluate(dtc_preds))

DTC
My binary classification accuracy: 0.8602618045815802

In [36]:
print('RFC')
print('My binary classification accuracy:', my_binary_eval.evaluate(rfc_preds))

RFC
My binary classification accuracy: 0.9860682561944834

In [37]:
print('GBT')
print('My binary classification accuracy:', my_binary_eval.evaluate(gbt_preds))

GBT
My binary classification accuracy: 0.9803646563814866